# Snow Elastic Modulus Calculations - Bergfeld et al. (2023) Method

This notebook demonstrates the application of the Bergfeld et al. (2023) elastic modulus parameterization to snowpit data. The Bergfeld method calculates elastic modulus from snow density using a power-law relationship optimized from Propagation Saw Test (PST) data.

The analysis uses the local snowpyt_mechparams package and snowpylot for CAAML parsing.


In [23]:
# Import Libraries
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Add the src directory to the path to import snowpyt_mechparams
sys.path.append('../src')
from snowpilot_utils import convert_grain_form, parse_sample_pits
from snowpyt_mechparams import density, elastic_modulus


Parse Snowpit Files


In [24]:
# Parse all snowpit files from the data folder
all_pits = parse_sample_pits('data')


Successfully parsed 50278 files
Failed to parse 0 files


Apply Bergfeld parameterization using density and associated uncertainty from:
1. Direct Measurement
2. Kim_geldsetzer

In [ ]:
# Collect relevant data from each snowpit
pit_info = []
layer_info = []

for pit in all_pits:
    pit_dict = {
        'pit_id': pit.core_info.pit_id,
        'layer_count': len(pit.snow_profile.layers),
    }
    pit_info.append(pit_dict)

    for layer in pit.snow_profile.layers:
        # Create base layer dictionary
        layer_dict = {
            'pit_id': pit.core_info.pit_id,
            'hand_hardness': layer.hardness,
            'grain_form_primary': layer.grain_form_primary,
        }

        # Add kim_geldsetzer grain form conversion if grain form data exists
        if layer.grain_form_primary:
            layer_dict['kim_geldsetzer_grain_form'] = convert_grain_form(layer.grain_form_primary, 'kim_geldsetzer')
            try: #calculate density using kim_geldsetzer method
                density_ufloat = density.calculate_density( 
                    method='kim_geldsetzer',
                    hand_hardness=layer.hardness,
                    grain_form=layer.kim_geldsetzer_grain_form
                )
                layer_dict['density_kim_geldsetzer'] = density_ufloat.nominal_value
                layer_dict['density_kim_geldsetzer_uncertainty'] = density_ufloat.std_dev
            except Exception:
                layer_dict['density_kim_geldsetzer'] = None
        else:
            layer_dict['kim_geldsetzer_grain_form'] = None
            layer_dict['density_kim_geldsetzer'] = None

        layer_info.append(layer_dict)

    for density_obs in pit.snow_profile.density_profile:
        for layer in pit.snow_profile.layers:
            if density_obs.depth_top == layer.depth_top and density_obs.thickness == layer.thickness: #same layer
                layer_dict['density_measured'] = density_obs.density[0] # nominal value of density
            else:
                layer_dict['density_measured'] = None

# Create a dataframe from the pit and layer info
pit_df = pd.DataFrame(pit_info)
layer_df = pd.DataFrame(layer_info)

layer_df.to_csv('layer_df.csv', index=False)


Apply Bergfeld Elastic Modulus Parameterization

Now we'll apply the Bergfeld et al. (2023) method to calculate elastic modulus for all layers that have density values.
